# Summaries of timing tests

We compare several optimizers, in both [R](http://www.R-project.org) and [Julia](http://julialang.org), fitting a selection of linear mixed models.  In __R__ the optimizers are called from `lmer` from the [lme4 package](https://github.com/lme4/lme4) (version 1.1-8). In __Julia__ the `lmm` function from the [MixedModels package](https://github.com/dmbates/MixedModels.jl) calls the optimizers.

There are differences in the model formulations in `lme4` and in `MixedModels`.  The numerical representation of the model in `lme4` and the method of evaluating the objective, described in [this paper](http://arxiv.org/abs/1406.5823), is the same for all models.  In `MixedModels` there are specialized representations for some model forms, such as models with a single grouping factor for the random effects.  Some of the specialized representations allow for evaluation of the gradient of the objects, which can enhance convergence (but, interestingly, sometimes can impede convergence).

## Methodology

To provide consistency we have copied all the data sets used in the timings to the `Timings` package itself.  We have done all timings on the same computer.  This computer has a relatively recent Intel processor and we used the [Intel Math Kernel Library (MKL)](https://software.intel.com/en-us/intel-mkl) with Julia.  We attempted to use [Revolution R Open (RRO)](www.revolutionanalytics.com/revolution-r-open) as the R implementation as it can be configured with MKL.  However, we ran into version problems with this so we used the standard Ubuntu version of R linked against OpenBLAS, which is also multi-threaded.

Variables were renamed in the pattern:
- __Y__ the response
- __A__, __B__, $\dots$ categorical covariates
- __G__, __H__, __I__, $\dots$ grouping factors for random effects
- __U__, __V__, $\dots$ (skipping __Y__) continuous covariates

The data are saved in [JSON (JavaScript Object Notation)](http://json.org) files in the directory accessible as
```r
system.file("JSON",package="Timings")
```
within R.  The directory name will end with `./Timings/inst/JSON/` in the package source directory, for example the result of cloning the [github repository](https://github.com/Stat990-033/Timings).

The `Timings` package for __R__ provides a `retime` function that takes the name of one of these JSON files and, optionally, the name of a file with the updated timings.  Similarly there are some source files for Julia retimings.


In [16]:
include("../julia/retime.jl")

retime (generic function with 2 methods)

In [17]:
retime("../JSON/Alfalfa.json","/tmp/Alfalfa.json")

dsname => "Alfalfa"
form => Formula: Y ~ 1 + B * A + (1 | G)
nopt => 1
mtype => PLSOne
-2log(likelihood) time(s) feval geval optimizer
      -10.8102     0.5050    15     0 bobyqa
      -10.8102     0.0220    36     0 Nelder_Mead
      -10.8102     0.0420    13     0 NLOPT_LN_BOBYQA
      -10.8102     0.0210    22     0 NLOPT_LN_COBYLA
      -10.8102     0.0210    24     0 NLOPT_LN_NELDERMEAD
      -10.8102     0.0220    36     0 NLOPT_LN_SBPLX
      -10.8102     0.1090    14     0 optimx:L-BFGS-B
      -10.8102     0.1290    19     0 optimx:nlminb
      -10.8102     0.2440    NA     0 optimx:spg
      -10.8102     0.1090    NA     0 optimx:bobyqa
      -10.8102     0.0017    15     0 LN_BOBYQA
      -10.8102     0.0021    25     0 LN_COBYLA
      -10.8102     0.0021    26     0 LN_NELDERMEAD
      -10.8102     0.0032    50     0 LN_SBPLX
      -10.8102     0.0016    14    14 LD_MMA
      -10.8102     0.0018    16    16 LD_CCSAQ
      -10.8102     0.0014    10     8 LD_SLSQP
      -10.

5005

The names of the optimizers used with `lmm` are those from the [NLopt package](https://github.com/JuliaOpt/NLopt.jl) for __Julia__.  Names that begin with `LD_` are gradient-based methods.  Names that begin with `LN_` are derivative-free methods.  There is one other derivative-free method, `LN_PRAXIS`, available in the `NLopt` package but, for some reason, it can hang on very simple problems like this.  Frequently we omit it.

The optimizers used with `lmer` include the `Nelder_Mead` optimizer built into the `lme4` package, the `bobyqa` optimizer from the [minqa package](http://cran.rstudio.com/web/packages/minqa/index.html), the derivative-free optimizers from the [nloptr package](http://cran.rstudio.com/web/packages/nloptr/index.html) and several optimizers from the [optimx pacakge](http://cran.rstudio.com/web/packages/optimx/index.html).

The `optimx:bobyqa` optimizer is just a wrapper around `bobyqa` (bounded optimization by quadratic approximation) from the `minqa` package and should provide results similar to those from the `bobyqa` optimizer.  For some reason the number of function evaluations is not reported for the version in `optimx`.

The optimizers from `nloptr` (i.e. those whose names begin with `NLOPT_LN_`) use the same underlying code as do the similarly named optimizers in the `NLopt` package for __Julia__.  The number of iterations to convergence should be similar for the same underlying code, although not nessarily exactly the same because the evaluation of the objective in __R__ and in __Julia__ may produce slightly different answers.  Also the convergence criteria in the __Julia__ version are more strict than those in the __R__ version

Also shown are the value of the criterion (negative twice the log-likelihood, lower is better) achieved, the elapsed time and the number of function and gradient evaluations.  The `nopt` value is the number of parameters in the optimization problem.  `mtype` is the model type in the __Julia__ code.  There are special methods for solving the penalized least squares (PLS) problem, and for evaluating the

 objective and its gradient when there is only one grouping factor for the random effects

The `Alfalfa` example is a particularly easy one and all of the optimizerws converge to an objective value close to -10.81023 in less than 0.6 seconds.

### Tabulating results

For the `Alfalfa` data there is not much of a burden in refitting the model with all the __Julia__ optimizers just to get the table shown above.  But other examples can take an hour or more to converge and we don't really need to refit them every time.  The `tabulate.jl` file contains a function `optdir` to create a `DataFrame` from the results of all the model fits.

In [22]:
include("../julia/tabulate.jl")
res = optdir("../JSON");
res[1:30,[1,2,3,6,8,9]]

,opt,dsname,n,np,times,devs
1,LD_CCSAQ,Alfalfa,72,1,0.001,-10.81
2,LD_CCSAQ,AvgDailyGain,32,1,0.001,9.971
3,LD_CCSAQ,AvgDailyGain,32,1,0.001,13.617
4,LD_CCSAQ,BIB,24,1,0.001,92.991
5,LD_CCSAQ,Bond,21,1,0.001,115.707
6,LD_CCSAQ,bs10,1104,20,1.138,1030.955
7,LD_CCSAQ,bs10,1104,8,0.041,1120.075
8,LD_CCSAQ,cake,270,1,0.003,1679.052
9,LD_CCSAQ,Cultivation,24,1,0.001,73.322
10,LD_CCSAQ,Demand,77,2,0.008,-257.409


In [23]:
res[res[:opt] .== "NLOPT_LN_BOBYQA",[1,2,3,6,8,9]]

,opt,dsname,n,np,times,devs
1,NLOPT_LN_BOBYQA,Alfalfa,72,1,0.042,-10.81
2,NLOPT_LN_BOBYQA,Animal,20,2,0.023,6.478
3,NLOPT_LN_BOBYQA,Assay,60,2,0.032,-240.389
4,NLOPT_LN_BOBYQA,AvgDailyGain,32,1,0.02,9.971
5,NLOPT_LN_BOBYQA,AvgDailyGain,32,1,0.02,13.617
6,NLOPT_LN_BOBYQA,BIB,24,1,0.02,92.991
7,NLOPT_LN_BOBYQA,Bond,21,1,0.02,115.707
8,NLOPT_LN_BOBYQA,bs10,1104,20,4.645,1030.955
9,NLOPT_LN_BOBYQA,bs10,1104,8,1.088,1080.081
10,NLOPT_LN_BOBYQA,cake,270,1,0.053,1679.052


## Proportion converged

The most important question regarding the optimizers is whether or not they have converged to the global optimum.  We cannot test this directly.  Instead we use a "crowd-sourced" criterion and evaluate the difference of the objective achieved by a particular algorithm and the minimum.  This is called the excess.

In [24]:
res[res[:opt] .== "NLOPT_LN_BOBYQA",[:opt,:dsname,:excess]]

,opt,dsname,excess
1,NLOPT_LN_BOBYQA,Alfalfa,4.1099568193203595e-11
2,NLOPT_LN_BOBYQA,Animal,9.302909553809968e-9
3,NLOPT_LN_BOBYQA,Assay,7.924335989173414e-6
4,NLOPT_LN_BOBYQA,AvgDailyGain,2.6911806116913795e-12
5,NLOPT_LN_BOBYQA,AvgDailyGain,0.0
6,NLOPT_LN_BOBYQA,BIB,8.27316029017311e-8
7,NLOPT_LN_BOBYQA,Bond,0.0
8,NLOPT_LN_BOBYQA,bs10,7.629679885212681e-6
9,NLOPT_LN_BOBYQA,bs10,3.4064009923895355e-7
10,NLOPT_LN_BOBYQA,cake,7.999005902092904e-10
